In [ ]:
import joblib

# Load the trained model
model_path = 'spam_classifier_model.pkl'
model = joblib.load(model_path)

# Predict function
def predict_spam(text):
    prediction = model.predict([text])[0]
    return 'Spam' if prediction else 'Not Spam'

# Example usage
if __name__ == "__main__":
    example_text = 'Hey Virgin Plus Member, new offer alert! Visit mb.vpc.ca/41bbxSS to get your first 4 months of RDS for just $59.97! ( vpc.ca/manageprefs )'
    print(f'Prediction: {predict_spam(example_text)}')


Prediction: Spam
